In [1]:
# # Mount the Google drive onto Colab Virtual Environment
# from google.colab import drive
# drive.mount('/content/drive')
root = '.'
# root = '/content/drive/My Drive/English Dataset'

In [2]:
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, Embedding, Dense, CuDNNLSTM, LSTM
from keras.models import Model
import tensorflow as tf
import pickle

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [0]:
with open(os.path.join(root, 'en-mar_cleaned.en')) as f:
    lines = f.readlines()
    
lines = pd.DataFrame(lines, columns=['Src'])
lines['Tar'] = None

with open(os.path.join(root, 'en-mar_cleaned.ma')) as f:
    lines['Tar'] = f.readlines()

In [0]:
lines.Src = lines.Src.apply(lambda x: x.strip())
lines.Tar = lines.Tar.apply(lambda x: x.strip())
lines.Tar = '<START> ' + lines.Tar + ' <END>'

In [41]:
lines.shape

(50000, 2)

In [42]:
lines.sample(10)

,Src,Tar
30563,copy cd dvd,<START> सीडी व डीवीडी की नक़ल करें <END>
4147,easthaven,<START> ईस्टहैवन <END>
27292,scroll pane,<START> स्क्रॉल फलक <END>
3059,golf,<START> गोल्फ <END>
37067,if you import them you will be able to see and...,<START> यदि आप उन्हें आयात करते हैं तो आप एक ब...
41266,whether the input cursor is visible,<START> क्या इनपुट संकेतक दिखाई देता है <END>
27277,menu bar,<START> मेन्यू पट्टी <END>
25783,plugin title,<START> प्लगइन शीर्षकः <END>
10200,select file to add,<START> डैरक्टरी चुनो <END>
35184,do you really want to quit,<START> क्या आप सचमुच बाहर होना चाहते हैं <END>


In [0]:
with open('/content/drive/My Drive/English Dataset/embeddings.en', 'rb') as f:
    src_summary = pickle.load(f)
    
with open('/content/drive/My Drive/English Dataset/embeddings.hi', 'rb') as f:
    tar_summary = pickle.load(f)

In [0]:
length_of_dict = len(tar_summary['dictionary'])
tar_summary['dictionary']['<START>'] = length_of_dict
tar_summary['reverse_dictionary'][length_of_dict] = '<START>'
tar_summary['embeddings'] = np.vstack((tar_summary['embeddings'], np.zeros((tar_summary['embeddings'].shape[1]))))

length_of_dict = len(tar_summary['dictionary'])
tar_summary['dictionary']['<END>'] = length_of_dict
tar_summary['reverse_dictionary'][length_of_dict] = '<END>'
tar_summary['embeddings'] = np.vstack((tar_summary['embeddings'], np.zeros((tar_summary['embeddings'].shape[1]))))

In [0]:
all_src_words = src_summary['dictionary'].keys()
all_tar_words = tar_summary['dictionary'].keys()

In [46]:
max_len_src = max([len(sentence.split(' ')) for sentence in lines.Src])
max_len_src

111

In [47]:
max_len_tar = max([len(sentence.split(' ')) for sentence in lines.Tar])
max_len_tar

114

In [48]:
num_encoder_tokens = len(all_src_words)
num_decoder_tokens = len(all_tar_words)
(num_encoder_tokens, num_decoder_tokens)

(100000, 100002)

In [49]:
lines = shuffle(lines)
lines.head(10)

,Src,Tar
34932,with,<START> के साथ <END>
17075,path,<START> पथ <END>
4856,ten of spades,<START> हुकुम का दहला <END>
12304,parameters,<START> पैरामीटर्सः <END>
12787,additional object files for this target,<START> अतिरिक्त वस्तु फ़ाइल के लिए यह लक्ष्य ...
21312,file manager,<START> फ़ाइल प्रबंधक <END>
37115,please insert a writable cd or dvd if you don ...,<START> कृपया एक लिखने योग्य सीडी या डीवीडी घु...
7742,a sample sdl project,<START> नमूना sdl प्रोजेक्ट <END>
40920,the rotation center on the x axis,<START> x अक्ष पर घुमाव कोण <END>
21557,revert a commit,<START> उलटें a <END>


In [0]:
def generate_batch(X, y, batch_size):
  while True:
    for j in range(0, len(X), batch_size):
      encoder_input_data = np.zeros((batch_size, max_len_src), dtype='float32')
      decoder_input_data = np.zeros((batch_size, max_len_tar), dtype='float32')
      decoder_target_data = np.zeros((batch_size, max_len_tar, num_decoder_tokens), dtype='float32')
      
      for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
        for t, word in enumerate(input_text.split()):
          if word not in src_summary['dictionary'].keys():
            word = 'UNK'
          encoder_input_data[i, t] = src_summary['dictionary'][word]
        
        for t, word in enumerate(target_text.split()):
          if word not in tar_summary['dictionary'].keys():
            word = 'UNK'
          
          if t < len(target_text.split())-1:
            decoder_input_data
            
          if t > 0:
            decoder_target_data[i, t-1, tar_summary['dictionary'][word]] = 1
            
        yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [51]:
for i in range(len(lines) // 1000):
  print('Set:', i+1)
  X, y = lines.Src[i * 1000:(i+1) * 1000], lines.Tar[i * 1000:(i+1) * 1000]
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
  
  latent_dim = 64
  
  train_samples = len(X_train)
  val_samples = len(X_test)
  batch_size = 16
  epochs = 5
  
  if i > 0:
    model.load_weights('/content/drive/My Drive/English Dataset/translator.h5')
  
  ### Encoder ###
  # Input layer
  encoder_inputs = Input(shape=(None, ), name='Encoder_Inputs')

  # Word2Vec embedding layer
  encoder_embedding_layer = Embedding(num_encoder_tokens, latent_dim, mask_zero=True, 
                                      weights=[src_summary['embeddings']], 
                                      name='English_Embeddings')
  encoder_embeddings = encoder_embedding_layer(encoder_inputs)

  # Encoder LSTM layer
  encoder_lstm = LSTM(64, return_state=True, name='Encoder_LSTM')
  encoder_outputs, state_h, state_c = encoder_lstm(encoder_embeddings)

  # Save states for decoder
  encoder_states = [state_h, state_c]


  ### Decoder ###
  # Input layer
  decoder_inputs = Input(shape=(None, ), name='Decoder_Inputs')

  # Word2Vec embedding layer
  decoder_embedding_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero=True, 
                                      weights=[tar_summary['embeddings']], 
                                      name='Hindi_Embeddings')
  decoder_embeddings = decoder_embedding_layer(decoder_inputs)

  # Decoder LSTM layer
  decoder_lstm = LSTM(64, return_sequences=True, return_state=True, name='Decoder_LSTM')
  decoder_outputs, _, _ = decoder_lstm(decoder_embeddings, initial_state=encoder_states)

  # Fully connected Softmax layer for predictions
  decoder_dense = Dense(num_decoder_tokens, activation='softmax', name='Dense')
  decoder_outputs = decoder_dense(decoder_outputs)


  ### Model ###
  model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
  model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])
  model.fit_generator(generator=generate_batch(X_train, y_train, batch_size=batch_size), 
                      steps_per_epoch=train_samples//batch_size, 
                      epochs=epochs, 
                      validation_data=generate_batch(X_test, y_test, batch_size=batch_size), 
                      validation_steps=val_samples//batch_size)

  model.save('/content/drive/My Drive/English Dataset/translator.h5')

Set: 1
Epoch 1/5


ResourceExhaustedError: ignored

In [0]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim, ))
decoder_state_input_c = Input(shape=(latent_dim, ))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2 = decoder_embedding_layer(decoder_inputs)

decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2
)

In [0]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = tar_summary['dictionary']['<START>']
    
    stop_condition = False
    decoded_sentence = ''
    
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = tar_summary['reverse_dictionary'][sampled_token_index]
        decoded_sentence += ' ' + sampled_char
        
        if (sampled_char == '<END>') or (len(decoded_sentence) > 50):
            stop_condition = True
            
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index
        
        states_value = [h, c]
    return decoded_sentence

In [0]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [37]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][7:-5])
print('Predicted Hindi Translation:', decoded_sentence[:-5])

Input English sentence: col lection
Actual Hindi Translation:  संग्रह 
Predicted Hindi Translation:  विवरण विवरण 


In [0]:
import json

with open("/var/log/colab-jupyter.log", "r") as fo:
  for line in fo:
    print(json.loads(line)['msg'])

Writing notebook server cookie secret to /root/.local/share/jupyter/runtime/notebook_cookie_secret
google.colab serverextension initialized.
Serving notebooks from local directory: /
0 active kernels
The Jupyter Notebook is running at:
http://172.28.0.2:9000/
Use Control-C to stop this server and shut down all kernels (twice to skip confirmation).
Kernel started: c4dbbf68-7cc1-4958-81c9-84080122875b
Adapting to protocol v5.1 for kernel c4dbbf68-7cc1-4958-81c9-84080122875b
Adapting to protocol v5.1 for kernel c4dbbf68-7cc1-4958-81c9-84080122875b
2019-04-01 14:36:25.486024: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-04-01 14:36:25.486495: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x19c2520 executing computations on platform Host. Devices:
2019-04-01 14:36:25.486530: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
2019-04-01 14:36:25.633812: I tensorflow/stream_exec